# @consumes basics

In [ ]:
# | hide

import asyncio
import platform

import asyncer
from IPython.display import Markdown as md

from fastkafka._components._subprocess import terminate_asyncio_process
from fastkafka._testing.apache_kafka_broker import run_and_match
from fastkafka.testing import ApacheKafkaBroker, run_script_and_cancel

You can use `@consumes` decorator to consume messages from Kafka topics. 

In this guide we will create a simple FastKafka app that will consume `HelloWorld` messages from hello_world topic.

## Import `FastKafka`

To use the `@consumes` decorator, first we need to import the base FastKafka app to create our application.

In [ ]:
# | echo: false

import_fastkafka = """from fastkafka import FastKafka
"""

md(f"```python\n{import_fastkafka}\n```")

```python
from fastkafka import FastKafka

```

In this demo we will log the messages to the output so that we can inspect and verify that our app is consuming properly. For that we need to import the logger.

In [ ]:
# | echo: false

import_logger = """from fastkafka._components.logger import get_logger

logger = get_logger(__name__)
"""

md(f"```python\n{import_logger}\n```")

```python
from fastkafka._components.logger import get_logger

logger = get_logger(__name__)

```

## Define the structure of the messages
Next, you need to define the structure of the messages you want to consume from the topic using [pydantic](https://docs.pydantic.dev/). For the guide we'll stick to something basic, but you are free to define any complex message structure you wish in your project, just make sure it can be JSON encoded.

Let's import `BaseModel` and `Field` from pydantic and create a simple `HelloWorld` class containing one string parameter `msg`

In [ ]:
# | echo: false

import_pydantic = """from pydantic import BaseModel, Field
"""
md(f"```python\n{import_pydantic}\n```")

```python
from pydantic import BaseModel, Field

```

In [ ]:
# | echo: false

define_HelloWorld = """
class HelloWorld(BaseModel):
    msg: str = Field(
        ...,
        example="Hello",
        description="Demo hello world message",
    )
"""
md(f"```python\n{define_HelloWorld}\n```")

```python

class HelloWorld(BaseModel):
    msg: str = Field(
        ...,
        example="Hello",
        description="Demo hello world message",
    )

```

## Create a base FastKafka app

Now we will create and define a base FastKafka app, replace the `<url_of_your_kafka_bootstrap_server>` and `<port_of_your_kafka_bootstrap_server>` with the actual values of your Kafka bootstrap server

In [ ]:
# | echo: false

create_app = """

kafka_brokers = {
    "demo_broker": {
        "url": "<url_of_your_kafka_bootstrap_server>",
        "description": "local demo kafka broker",
        "port": "<port_of_your_kafka_bootstrap_server>",
    }
}

app = FastKafka(kafka_brokers=kafka_brokers)
"""
md(f"```python\n{create_app}\n```")

```python


kafka_brokers = {
    "demo_broker": {
        "url": "<url_of_your_kafka_bootstrap_server>",
        "description": "local demo kafka broker",
        "port": "<port_of_your_kafka_bootstrap_server>",
    }
}

app = FastKafka(kafka_brokers=kafka_brokers)

```

## Create a consumer function and decorate it with `@consumes`

Let's create a consumer function that will consume `HelloWorld` messages from *hello_world* topic and log them.

In [ ]:
# | echo: false

decorate_consumes = """@app.consumes()
async def on_hello_world(msg: HelloWorld):
    logger.info(f"Got msg: {msg}")
"""
md(f"```python\n{decorate_consumes}\n```")

```python
@app.consumes()
async def on_hello_world(msg: HelloWorld):
    logger.info(f"Got msg: {msg}")

```

The function decorated with the `@consumes` decorator will be called when a message is produced to Kafka.

The message will then be injected into the typed *msg* argument of the function and its type will be used to parse the message.

In this example case, when the message is sent into a *hello_world* topic, it will be parsed into a HelloWorld class and `on_hello_world` function will be called with the parsed class as *msg* argument value.

## Final app

Your app code should look like this:

In [ ]:
# | echo: false

consumes_example = (
    import_fastkafka
    + import_pydantic
    + import_logger
    + define_HelloWorld
    + create_app
    + decorate_consumes
)
md(f"```python\n{consumes_example}\n```")

```python
from fastkafka import FastKafka
from pydantic import BaseModel, Field
from fastkafka._components.logger import get_logger

logger = get_logger(__name__)

class HelloWorld(BaseModel):
    msg: str = Field(
        ...,
        example="Hello",
        description="Demo hello world message",
    )


kafka_brokers = {
    "demo_broker": {
        "url": "<url_of_your_kafka_bootstrap_server>",
        "description": "local demo kafka broker",
        "port": "<port_of_your_kafka_bootstrap_server>",
    }
}

app = FastKafka(kafka_brokers=kafka_brokers)
@app.consumes()
async def on_hello_world(msg: HelloWorld):
    logger.info(f"Got msg: {msg}")

```

## Run the app

In [ ]:
# | echo: false

script_file = "consumer_example.py"
filename = script_file.split(".py")[0]
cmd = f"fastkafka run --num-workers=1 --kafka-broker=demo_broker {filename}:app"
md(
    f"Now we can run the app. Copy the code above in {script_file} and run it by running\n```shell\n{cmd}\n```"
)

Now we can run the app. Copy the code above in consumer_example.py and run it by running
```shell
fastkafka run --num-workers=1 --kafka-broker=demo_broker consumer_example:app
```

After running the command, you should see this output in your terminal:

In [ ]:
# | hide

with ApacheKafkaBroker(
    topics=["hello_world"], apply_nest_asyncio=True
) as bootstrap_server:
    server_url = bootstrap_server.split(":")[0]
    server_port = bootstrap_server.split(":")[1]
    exit_code, output = await run_script_and_cancel(
        script=consumes_example.replace(
            "<url_of_your_kafka_bootstrap_server>", server_url
        ).replace("<port_of_your_kafka_bootstrap_server>", server_port),
        script_file=script_file,
        cmd=cmd,
        cancel_after=5,
    )

    expected_returncode = 1 if platform.system() == "Windows" else 0
    assert exit_code == expected_returncode, output.decode("UTF-8")

23-06-02 12:08:39.473 [INFO] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): entering...
23-06-02 12:08:39.481 [WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): (<_WindowsSelectorEventLoop running=True closed=False debug=False>) is already running!
23-06-02 12:08:39.481 [WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): calling nest_asyncio.apply()
23-06-02 12:08:39.481 [INFO] fastkafka._components.test_dependencies: Java is already installed.
23-06-02 12:08:39.489 [INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
23-06-02 12:08:39.489 [INFO] fastkafka._components.test_dependencies: Kafka is installed.
23-06-02 12:08:39.497 [INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
23-06-02 12:08:39.497 [INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...

23-06-02 12:08:39.497 [INFO] fastkafka._testing.apache_kafka_broker: zookeeper s

ValueError: Could not start zookeeper with params: [{'zookeeper_port': 2181}, {'zookeeper_port': '50568'}, {'zookeeper_port': '50569'}, {'zookeeper_port': '50570'}]

In [ ]:
# | echo: false

print(output.decode("UTF-8"))

## Send the message to kafka topic

Lets send a `HelloWorld` message to the *hello_world* topic and check if our consumer kafka application has logged the received message. In your terminal, run:

In [ ]:
# | echo: false

script_extension = ".bat" if platform.system() == "Windows" else ".sh"
escape_char = "^" if platform.system() == "Windows" else ".\\"

kafka_msg = 'echo {{ {escape_char}"msg{escape_char}": {escape_char}"Hello world{escape_char}" }}'.format(escape_char=escape_char)

producer_cmd = f'{kafka_msg} | kafka-console-producer'+script_extension+' --topic=hello_world --bootstrap-server=<addr_of_your_kafka_bootstrap_server>'
md(f"```shell\n{producer_cmd}\n```")

In [ ]:
# | hide


consumes_example = (
    import_fastkafka
    + import_pydantic
    + import_logger
    + define_HelloWorld
    + create_app
    + decorate_consumes
)

with ApacheKafkaBroker(
    topics=["hello_world"], apply_nest_asyncio=True
) as bootstrap_server:
    async with asyncer.create_task_group() as task_group:
        server_url = bootstrap_server.split(":")[0]
        server_port = bootstrap_server.split(":")[1]
        consumer_task = task_group.soonify(run_script_and_cancel)(
            script=consumes_example.replace(
                "<url_of_your_kafka_bootstrap_server>", server_url
            ).replace("<port_of_your_kafka_bootstrap_server>", server_port),
            script_file=script_file,
            cmd=cmd,
            cancel_after=20,
        )
        await asyncio.sleep(10)

        producer_task = task_group.soonify(asyncio.create_subprocess_shell)(
            cmd=producer_cmd.replace(
                "<addr_of_your_kafka_bootstrap_server>", bootstrap_server
            ),
            stdout=asyncio.subprocess.PIPE,
            stderr=asyncio.subprocess.PIPE,
        )

assert "Got msg: msg='Hello world'" in consumer_task.value[1].decode("UTF-8")

In [ ]:
print(consumer_task.value[1].decode("UTF-8"))

You should see the "Got msg: msg=\'Hello world\'" being logged by your consumer.

## Choosing a topic

You probably noticed that you didn't define which topic you are receiving the message from, this is because the `@consumes` decorator determines the topic by default from your function name.
The decorator will take your function name and strip the default "on_" prefix from it and use the rest as the topic name. In this example case, the topic is *hello_world*.

You can choose your custom prefix by defining the `prefix` parameter in consumes decorator, like this:

In [ ]:
# | echo: False

decorate_consumes_prefix = """
@app.consumes(prefix="read_from_")
async def read_from_hello_world(msg: HelloWorld):
    logger.info(f"Got msg: {msg}")
"""
md(f"```python\n{decorate_consumes_prefix}\n```")

In [ ]:
# | hide


consumes_example = (
    import_fastkafka
    + import_pydantic
    + import_logger
    + define_HelloWorld
    + create_app
    + decorate_consumes_prefix
)

with ApacheKafkaBroker(
    topics=["hello_world"], apply_nest_asyncio=True
) as bootstrap_server:
    async with asyncer.create_task_group() as task_group:
        server_url = bootstrap_server.split(":")[0]
        server_port = bootstrap_server.split(":")[1]
        consumer_task = task_group.soonify(run_script_and_cancel)(
            script=consumes_example.replace(
                "<url_of_your_kafka_bootstrap_server>", server_url
            ).replace("<port_of_your_kafka_bootstrap_server>", server_port),
            script_file=script_file,
            cmd=cmd,
            cancel_after=20,
        )
        await asyncio.sleep(10)

        producer_task = task_group.soonify(asyncio.create_subprocess_shell)(
            cmd=producer_cmd.replace(
                "<addr_of_your_kafka_bootstrap_server>", bootstrap_server
            ),
            stdout=asyncio.subprocess.PIPE,
            stderr=asyncio.subprocess.PIPE,
        )

assert "Got msg: msg='Hello world'" in consumer_task.value[1].decode("UTF-8")

Also, you can define the topic name completely by defining the `topic` in parameter in consumes decorator, like this:

In [ ]:
# | echo: false

decorate_consumes_topic = """
@app.consumes(topic="my_special_topic")
async def on_hello_world(msg: HelloWorld):
    logger.info(f"Got msg: {msg}")
"""
md(f"```python\n{decorate_consumes_topic}\n```")

In [ ]:
# | hide


consumes_example = (
    import_fastkafka
    + import_pydantic
    + import_logger
    + define_HelloWorld
    + create_app
    + decorate_consumes_topic
)

with ApacheKafkaBroker(
    topics=["my_special_topic"], apply_nest_asyncio=True
) as bootstrap_server:
    async with asyncer.create_task_group() as task_group:
        server_url = bootstrap_server.split(":")[0]
        server_port = bootstrap_server.split(":")[1]
        consumer_task = task_group.soonify(run_script_and_cancel)(
            script=consumes_example.replace(
                "<url_of_your_kafka_bootstrap_server>", server_url
            ).replace("<port_of_your_kafka_bootstrap_server>", server_port),
            script_file=script_file,
            cmd=cmd,
            cancel_after=20,
        )
        await asyncio.sleep(10)

        producer_task = task_group.soonify(asyncio.create_subprocess_shell)(
            cmd=producer_cmd.replace(
                "<addr_of_your_kafka_bootstrap_server>", bootstrap_server
            ).replace("topic=hello_world", "topic=my_special_topic"),
            stdout=asyncio.subprocess.PIPE,
            stderr=asyncio.subprocess.PIPE,
        )

assert "Got msg: msg='Hello world'" in consumer_task.value[1].decode(
    "UTF-8"
), consumer_task.value[1].decode("UTF-8")

## Message data

The message received from kafka is translated from binary JSON representation int the class defined by typing of *msg* parameter in the function decorated by the `@consumes` decorator.

In this example case, the message will be parsed into a `HelloWorld` class.

## Message metadata

If you need any of Kafka message metadata such as timestamp, partition or headers you can access the metadata by adding a EventMetadata typed argument to your consumes function and the metadata from the incoming message will be automatically injected when calling the consumes function.

Let's demonstrate that.

### Create a consumer function with metadata

The only difference from the original basic consume function is that we are now passing the `meta: EventMetadata` argument to the function. The `@consumes` decorator will register that and, when a message is consumed, it will also pass the metadata to your function. Now you can use the metadata in your consume function. 
Lets log it to see what it contains.

First, we need to import the EventMetadata

In [ ]:
# | echo: false

import_event_metadata = """from fastkafka import EventMetadata
"""
md(f"```python\n{import_event_metadata}\n```")

Now we can add the `meta` argument to our consuming function.

In [ ]:
# | echo: false

decorate_consumes_meta = """@app.consumes()
async def on_hello_world(msg: HelloWorld, meta: EventMetadata):
    logger.info(f"Got metadata: {meta}")
"""
md(f"```python\n{decorate_consumes_meta}\n```")

Your final app should look like this:

In [ ]:
# | echo: false

consumes_example = (
    import_fastkafka
    + import_pydantic
    + import_event_metadata
    + import_logger
    + define_HelloWorld
    + create_app
    + decorate_consumes_meta
)

md(f"```python\n{consumes_example}\n```")

Now lets run the app and send a message to the broker to see the logged message metadata.

In [ ]:
# | hide


with ApacheKafkaBroker(
    topics=["hello_world"], apply_nest_asyncio=True
) as bootstrap_server:
    async with asyncer.create_task_group() as task_group:
        server_url = bootstrap_server.split(":")[0]
        server_port = bootstrap_server.split(":")[1]
        consumer_task = task_group.soonify(run_script_and_cancel)(
            script=consumes_example.replace(
                "<url_of_your_kafka_bootstrap_server>", server_url
            ).replace("<port_of_your_kafka_bootstrap_server>", server_port),
            script_file=script_file,
            cmd=cmd,
            cancel_after=20,
        )
        await asyncio.sleep(10)

        producer_task = task_group.soonify(asyncio.create_subprocess_shell)(
            cmd=producer_cmd.replace(
                "<addr_of_your_kafka_bootstrap_server>", bootstrap_server
            ),
            stdout=asyncio.subprocess.PIPE,
            stderr=asyncio.subprocess.PIPE,
        )

assert "Got metadata: " in consumer_task.value[1].decode(
    "UTF-8"
), consumer_task.value[1].decode("UTF-8")

You should see a similar log as the one below and the metadata being logged in your app.

In [ ]:
# | echo: false

print(consumer_task.value[1].decode("UTF-8"))

As you can see in the log, from the metadata you now have the information about the partition, offset, timestamp, key and headers. :tada:

## Dealing with high latency consuming functions

If your functions have high latency due to, for example, lengthy database calls you will notice a big decrease in performance. This is due to the issue of how the consumes decorator executes your consume functions when consumeing events. By default, the consume function will run the consuming funtions for one topic sequentially, this is the most straightforward approach and results with the least amount of overhead.

But, to handle those high latency tasks and run them in parallel, FastKafka has a `DynamicTaskExecutor` prepared for your consumers. This executor comes with additional overhead, so use it only when you need to handle high latency functions.

Lets demonstrate how to use it.

In [ ]:
decorate_consumes_executor = """@app.consumes(executor="DynamicTaskExecutor")
async def on_hello_world(msg: HelloWorld):
    logger.info(f"Got msg: {msg}")
"""
md(f"```python\n{decorate_consumes}\n```")

Lets send a `HelloWorld` message to the *hello_world* topic and check if our consumer kafka application has logged the received message. In your terminal, run:

In [ ]:
# | echo: false

script_extension = ".bat" if platform.system() == "Windows" else ".sh"
escape_char = "^" if platform.system() == "Windows" else ".\\"

kafka_msg = 'echo {{ {escape_char}"msg{escape_char}": {escape_char}"Hello world{escape_char}" }}'.format(escape_char=escape_char)


producer_cmd = f'{kafka_msg} | kafka-console-producer'+script_extension+' --topic=hello_world --bootstrap-server=<addr_of_your_kafka_bootstrap_server>'
md(f"```shell\n{producer_cmd}\n```")

In [ ]:
# | hide


consumes_example = (
    import_fastkafka
    + import_pydantic
    + import_logger
    + define_HelloWorld
    + create_app
    + decorate_consumes_executor
)

with ApacheKafkaBroker(
    topics=["hello_world"], apply_nest_asyncio=True
) as bootstrap_server:
    async with asyncer.create_task_group() as task_group:
        server_url = bootstrap_server.split(":")[0]
        server_port = bootstrap_server.split(":")[1]
        consumer_task = task_group.soonify(run_script_and_cancel)(
            script=consumes_example.replace(
                "<url_of_your_kafka_bootstrap_server>", server_url
            ).replace("<port_of_your_kafka_bootstrap_server>", server_port),
            script_file=script_file,
            cmd=cmd,
            cancel_after=20,
        )
        await asyncio.sleep(10)

        producer_task = task_group.soonify(asyncio.create_subprocess_shell)(
            cmd=producer_cmd.replace(
                "<addr_of_your_kafka_bootstrap_server>", bootstrap_server
            ),
            stdout=asyncio.subprocess.PIPE,
            stderr=asyncio.subprocess.PIPE,
        )

assert "Got msg: msg='Hello world'" in consumer_task.value[1].decode("UTF-8")

You should see the "Got msg: msg=\'Hello world\'" being logged by your consumer.